# Задание 2

### Инициализация окружения
В качестве документации для задания 2 выступает данный документ Jupyter Notebook. В нём записаны все скрипты и SQL-запросы, касающиеся пунктов 2.1 и 2.2.

venv для данного задания и задания 3 одинаковый, поэтому на данном этапе можно задействовать некоторые из необходимых далее библиотек.

In [ ]:
%pip install sqlalchemy psycopg2-binary pandas

Далее следует создать соединение к базе данных, через которое будут протестированы все последующие запросы.
Необходимые данные для соединения с БД должны браться хотя бы из .env-файла, но в рамках Jupyter Notebook все параметры внесены в отдельный словарь.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Параметры подключения к БД
DB_CONFIG = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'main_db'
}

# Создаём движок SQLAlchemy
engine = create_engine(
            f"{DB_CONFIG['drivername']}://{DB_CONFIG['username']}:{DB_CONFIG['password']}@"
            f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        )
# Создаём соединение с БД на основе движка
db_connection = engine.connect()
db_connection.closed # Если выведен False, то соединение настроено

False

### Задание 2.1.
Для данного задания необходимо получить информацию о сумме товаров, заказанных под каждого клиента.

Получение суммы реализовано через последовательность запросов JOIN вместе с функцией COALESCE, позволяющей вывести сумму 0 в случае отсутствия заказов у данного клиента.
Вместо обычного JOIN используется LEFT JOIN для включения в вывод клиентов, которые ещё не сделали ни одного заказа.

In [14]:
task_2_1_query = """
SELECT 
    c.name AS client_name,
    COALESCE(SUM(og.amount), 0) AS total_amount
FROM Clients c
LEFT JOIN Orders o ON o.client_id = c.id
LEFT JOIN Ordered_goods og ON og.order_id = o.id
LEFT JOIN Goods g ON g.id = og.good_id
GROUP BY c.id, c.name
ORDER BY total_amount DESC;
"""

# Выполняем запрос на получения данных из БД
df = pd.read_sql(task_2_1_query, engine)
df

,client_name,total_amount
0,Козлова Ирина Петровна,29
1,Кузнецов Алексей Викторович,29
2,Захаров Сергей Викторович,29
3,Петров Петр Петрович,23
4,Смирнова Ольга Владимировна,20
5,Иванов Иван Иванович,17
6,Волков Андрей Сергеевич,13
7,Сидорова Мария Сергеевна,8
8,Николаева Екатерина Александровна,2
9,Федоров Дмитрий Николаевич,0


При этом может потребоваться информация о суммарной стоимости всех заказанных товарах, которое получается простой заменой выражения внутри функции SUM.

In [15]:
# Заменяем выражение внутри функции SUM
sum_price_query = task_2_1_query.replace("SUM(og.amount)", "SUM(g.price * og.amount)")

# Заменяем все упоминания total_amount на total_cost
sum_price_query = sum_price_query.replace("total_amount", "total_cost")

df = pd.read_sql(sum_price_query, engine)
df

,client_name,total_cost
0,Козлова Ирина Петровна,1817971.9
1,Кузнецов Алексей Викторович,1794971.9
2,Захаров Сергей Викторович,1731471.9
3,Петров Петр Петрович,1335977.9
4,Смирнова Ольга Владимировна,1187480.9
5,Иванов Иван Иванович,830484.8
6,Волков Андрей Сергеевич,804987.9
7,Сидорова Мария Сергеевна,449992.0
8,Николаева Екатерина Александровна,128998.0
9,Федоров Дмитрий Николаевич,0.0


### Задание 2.2.
Необходимо написать запрос на поиск количества дочерних элементов первого уровня вложенности для категорий номенклатуры.

Поскольку при выполнении данного задания используется PostgreSQL с расширением ltree, в запросах применяются соответствующие инструменты поиска элементов по дереву. Хотя это решение не универсально среди всех реляционных СУБД, в данных условиях оно наиболее оптимальное.

***Комментарии к запросу***
1. *LEFT JOIN* используется, опять же, для вывода элементов первого уровня без дочерних подэлементов.
2. Условие "*ON c2.path <@ c1.path*" возвращает True в случае иерархической принадлежности текущего элемента из таблицы *Catalogue c2* данному корневому элементу из *c1*.
3. Условие "*AND nlevel(c2.path) = nlevel(c1.path) + 1*" возвращает True, если текущий элемент из *c2* является подэлементом первого уровня вложенности для данной категории из *c1*.

In [23]:
task_2_2_query = """
SELECT 
    c1.id,
    c1.name,
    COUNT(c2.id) AS num_of_children
FROM Catalogue c1
LEFT JOIN Catalogue c2
          ON c2.path <@ c1.path
          AND nlevel(c2.path) = nlevel(c1.path) + 1
GROUP BY c1.id, c1.name
ORDER BY num_of_children DESC;
"""

df = pd.read_sql(task_2_2_query, engine)
df

,id,name,num_of_children
0,1,Бытовая техника,3
1,8,Ноутбуки,2
2,2,Компьютеры,2
3,4,Холодильники,2
4,11,Моноблоки,0
5,9,17',0
6,5,однокамерные,0
7,3,Стиральные машины,0
8,10,19',0
9,6,двухкамерные,0


### Задание 2.3.1.
Требуется написать VIEW для вывода топ-5 самых покупаемых товаров по количеству штук в заказах.

Для отчёта названия атрибутов итоговой таблицы приведены на русском.
Категорию 1-го уровня для данного товара с помощью ltree получить достаточно просто: необходимо разделить всю строку по знаку '.' и взять первый элемент массива как наименование корневой категории.
Однако, поскольку изначально в path для краткости хранится последовательность ID, надо предварительно выделить из неё конкретное наименование категории.

Поскольку требуется выделить топ-5 элементов, в данном запросе для оптимизации используется ограничение *LIMIT 5*. Однако если на одном из мест будет несколько претендентов, всё равно выводятся только 5 позиций, что может быть нежелательно в некоторых случаях.

In [74]:
# Для выделения названия категории пришлось добавить новый JOIN
# Так как на уровне проектирования индексы добавлены не были,
# выполнение данного VIEW на высоконагруженной системе может сильно замедлить работу БД

view_task_query = """
CREATE OR REPLACE VIEW top5_purchased_goods AS
SELECT 
    g.name AS Наименование_товара,
    root_catalogue.name AS Категория_1_го_уровня,
    SUM(og.amount) AS Общее_количество_проданных_штук
FROM Goods g
JOIN Ordered_goods og ON g.id = og.good_id
JOIN Catalogue c ON g.catalogue_id = c.id
JOIN Catalogue root_catalogue ON root_catalogue.id = (split_part(c.path::text, '.', 1))::integer
GROUP BY g.id, g.name, root_catalogue.name
ORDER BY Общее_количество_проданных_штук DESC
LIMIT 5;
"""

Устранить данный недостаток можно, используя оконную функцию *DENSE RANK()* (функция *RANK* может быть нежелательна, так как разрывает порядок ранжирования), однако это несколько усложнит и затормозит VIEW-запрос, в связи с чем его следует применять только при явном объявлении об этом в бизнес-требованиях или ТЗ.

In [71]:
# Дополнительные операторы NULLIF и ::integer понадобились
# для преобразования varchar из path в integer и сравнения с ID

extended_view_task_query = """
CREATE OR REPLACE VIEW top5_purchased_goods AS
WITH goods_sales AS (
    SELECT 
        g.id AS goods_id,
        g.name AS goods_name,
        NULLIF(split_part(c.path::text, '.', 1), '')::integer AS root_category_id,
        SUM(og.amount) AS total_sold
    FROM Goods g
    LEFT JOIN Ordered_goods og ON g.id = og.good_id
    LEFT JOIN Catalogue c ON g.catalogue_id = c.id
    GROUP BY g.id, g.name, c.path
),
top5 AS (
    SELECT 
        goods_id,
        goods_name,
        root_category_id,
        total_sold,
        DENSE_RANK() OVER (ORDER BY total_sold DESC) AS position
    FROM goods_sales
)
SELECT 
    t5.goods_name AS Наименование_товара,
    cat.name AS Категория_1_го_уровня,
    t5.total_sold AS Общее_количество_проданных_штук
FROM top5 t5
LEFT JOIN Catalogue cat ON t5.root_category_id = cat.id
WHERE t5.position <= 5
ORDER BY t5.total_sold DESC, t5.goods_name;
"""

Соответственно, поскольку в ТЗ явно не прописано использование более сложной версии, в дальнейшем предполаегется использование варианта с применением *LIMIT 5*.

In [75]:
from sqlalchemy import text

db_connection.execute(text(view_task_query))
db_connection.commit()

df = pd.read_sql("SELECT * FROM top5_purchased_goods;", engine)
df

,Наименование_товара,Категория_1_го_уровня,Общее_количество_проданных_штук
0,"Ноутбук Lenovo 17""",Компьютеры,16
1,Телевизор LG,Бытовая техника,15
2,Телевизор Sony,Бытовая техника,14
3,"Ноутбук Dell 17""",Компьютеры,14
4,Холодильник Samsung,Бытовая техника,13


### Задание 2.3.2.
Необходимо проанализировать созданный запрос и саму структуру БД, а также оптимизировать её под увеличивающуюся нагрузку.

Для начала следует определить оптимальность выполнения запроса из задания 2.3.1.